In [ ]:
import boto3
import s3fs
import sagemaker
from sagemaker import get_execution_role
import time
import numpy as np
import pandas as pd
import json
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import bs4 as bs
import pickle
import requests
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score


In [ ]:
# session, role, bucket
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()             # IAM role to use by SageMaker

s3_bucket = sagemaker_session.default_bucket()
s3_prefix = 'MLEND-Capstone-Project'    

region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output".format(s3_bucket, s3_prefix)

image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")


In [ ]:
stock_data_preprocessed = pd.read_csv('stock_data_preprocessed.csv',parse_dates=True, index_col=[0,1])
print("-1: ", stock_data_preprocessed[stock_data_preprocessed['target']==-1].size)
print(" 0: ", stock_data_preprocessed[stock_data_preprocessed['target']==0].size)
print(" 1: ", stock_data_preprocessed[stock_data_preprocessed['target']==1].size)
#stock_data_preprocessed.head(10)

In [ ]:
# we use 2 hour frequency for the time series
freq = 'D'

# we predict for 7 days
prediction_length = 1

# we also use 7 days as context length, this is the number of state updates accomplished before making predictions
context_length = 50

end_training = pd.Timestamp("2018-12-31 00:00:00", freq=freq)

timeseries = []
for ID,ticker in list(enumerate(tickers)):
    ticker = stock_data_preprocessed.loc[(slice(None), ticker), :]
    if ticker.index[0][0]<end_training:
        timeseries.append(ticker)


In [111]:
training_data = [
    {
            "start": str(ts.index[0][0]),
            "target": ts['target'][ts.index[0][0]:end_training].tolist(), # We use -1, because pandas indexing includes the upper bound 
            "dynamic_feat": ts[['Adj Close','Volume']][ts.index[0][0]:end_training].values.T.tolist()
    }
    for ts in timeseries
]
print(len(training_data))

491


In [112]:
num_test_windows = 10

test_data = [
    {
        "start": str(ts.index[0][0]),
        "target": ts['target'][ts.index[0][0]:end_training + (2*k * prediction_length)].tolist(),
        "dynamic_feat": ts[['Adj Close','Volume']][ts.index[0][0]:end_training + (2*k * prediction_length)].values.T.tolist()
    }
    for k in range(1, num_test_windows + 1) 
    for ts in timeseries
]
print(len(test_data))


/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`


4910


In [113]:
def write_json_dataset(filename, data): 
    with open(filename, 'wb') as f:
        # for each of our times series, there is one JSON line
        for d in data:
            json_line = json.dumps(d) + '\n'
            json_line = json_line.encode('utf-8')
            f.write(json_line)            

In [115]:
%%time
write_json_dataset("train.json", training_data)
write_json_dataset("test.json", test_data)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


In [18]:
s3 = boto3.resource('s3')
def copy_to_s3(local_file, s3_path, override=False):
    assert s3_path.startswith('s3://')
    split = s3_path.split('/')
    bucket = split[2]
    path = '/'.join(split[3:])
    buk = s3.Bucket(bucket)
    
    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print('File s3://{}/{} already exists.\nSet override to upload anyway.\n'.format(s3_bucket, s3_path))
            return
        else:
            print('Overwriting existing file')
    with open(local_file, 'rb') as data:
        print('Uploading file to {}'.format(s3_path))
        buk.put_object(Key=path, Body=data)

In [20]:
%%time
copy_to_s3("train.json", s3_data_path + "/train/train.json")
copy_to_s3("test.json", s3_data_path + "/test/test.json")

Uploading file to s3://sagemaker-us-east-2-017500148529/MLEND-Capstone-Project/data/train/train.json
Uploading file to s3://sagemaker-us-east-2-017500148529/MLEND-Capstone-Project/data/test/test.json
CPU times: user 2.82 s, sys: 311 ms, total: 3.13 s
Wall time: 7.01 s


In [116]:
s3filesystem = s3fs.S3FileSystem()
with s3filesystem.open(s3_data_path + "/train/train.json", 'rb') as fp:
    print(fp.readline().decode("utf-8")[:100] + "...")

{"start": "2010-01-04 00:00:00", "target": [-1, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...


In [117]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.2xlarge',
    base_job_name='MLEND-Capstone-Project',
    output_path=s3_output_path
)

In [121]:
hyperparameters = {
    "time_freq": freq,
    "epochs": "100",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "num_dynamic_feat": 'auto',
}

In [122]:
estimator.set_hyperparameters(**hyperparameters)

In [ ]:
%%time
data_channels = {
    "train": s3_data_path + "/train/train.json",
    "test": s3_data_path + "/test/test.json"
}

estimator.fit(inputs=data_channels, wait=True)

2020-06-01 21:26:26 Starting - Starting the training job...
2020-06-01 21:26:28 Starting - Launching requested ML instances......
2020-06-01 21:27:31 Starting - Preparing the instances for training...
2020-06-01 21:28:19 Downloading - Downloading input data...
2020-06-01 21:28:45 Training - Downloading the training image..Arguments: train
[06/01/2020 21:29:07 INFO 140176366790464] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'num_dynamic_feat': u'auto', u'dropout_rate': u'0.10', u'mini_batch_size': u'128', u'test_quantiles': u'[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'num_eval_samples': u'100', u'learning_rate': u'0.001', u'num_cells': u'40', u'num_layers': u'2', u'embedding_dimension': u'10', u'_kvstore': u'auto', u'_num_kv_servers': u'auto', u'cardinality': u'auto', u'likelihood': u'student-t', u'early_stopping_patience': u''}
[06/01/2020 21:29:07 I

In [125]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    content_type="application/json")

---------------!

In [128]:
def encode_request(instance, num_samples, quantiles):
        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles"],
            "quantiles": quantiles
        }
        
        http_request_data = {
            "instances": [instance],
            "configuration": configuration
        }
        
        return json.dumps(http_request_data).encode('utf-8')


In [ ]:
#predictor = sagemaker.predictor.RealTimePredictor(endpoint='MLEND-Capstone-Project-2020-05-31-23-12-35-007')

In [129]:
def get_stock_prediction(ticker,date):
    try:
        date_pred = pd.Timestamp(date, freq='D')
        date_start = date_pred-50
        pred_df = stock_data_preprocessed.loc[(slice(str(date_start),str(date_pred)), ticker), :]
        result_df = pred_df.loc[(slice(str(date_pred),str(date_pred)), ticker), :]

        pred = {
                "start": str(date_pred),
                "target": pred_df['target'][date_start:date_pred-1].tolist(),
                "dynamic_feat": pred_df[['Adj Close','Volume']][date_start:date_pred].values.T.tolist()
        }

        req = encode_request(instance=pred, num_samples=50, quantiles=['0.1', '0.5', '0.9'])
        res = predictor.predict(req)

        prediction_data = json.loads(res.decode('utf-8'))
        pred = round(prediction_data['predictions'][0]['quantiles']['0.5'][0])
        result_df['prediction'] = pred
        return result_df
    except:
        print('{} did not trade today.'.format(ticker))

In [130]:
get_stock_prediction('AAPL', '2019-01-23')

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,,Adj Close,Volume,target,prediction
Date,Ticker,,,,
2019-01-23,AAPL,0.03886,0.026907,1,1


In [131]:
date_index = pd.read_csv('date_index.csv')
date_index = date_index.values.reshape(252).tolist()

def get_prediction_accuracy(ticker, date_range):
    ticker = str(ticker)
    i = 0
    target = []
    prediction = []

    for date in date_range:
        target.append(get_stock_prediction(ticker, date)['target'].values[0])
        prediction.append(int(get_stock_prediction(ticker, date)['prediction'].values[0]))
    target = list(np.array(target).reshape(252))
    prediction = list(np.array(prediction).reshape(252))
    data = {'target': list(target), 'prediction': list(prediction)}
    prediction_df = pd.DataFrame(data=data,index=date_index, columns=['target','prediction'])
    
    return accuracy_score(target, prediction), prediction_df

In [132]:
get_prediction_accuracy('AAPL', date_index)

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(0.746031746031746,             target  prediction
 2019-01-02      -1          -1
 2019-01-03       1          -1
 2019-01-04       1           1
 2019-01-07       1           1
 2019-01-08       1           1
 2019-01-09       0           1
 2019-01-10      -1           0
 2019-01-11       1          -1
 2019-01-14       1           1
 2019-01-15       1           1
 2019-01-16       0           1
 2019-01-17       0           0
 2019-01-18      -1           0
 2019-01-22       1          -1
 2019-01-23       1           1
 2019-01-24       1           1
 2019-01-25       1           1
 2019-01-28       1           1
 2019-01-29       1           1
 2019-01-30       1           1
 2019-01-31       1           1
 2019-02-01       1           1
 2019-02-04       0           1
 2019-02-05       0           0
 2019-02-06       0           0
 2019-02-07       0           0
 2019-02-08       0           0
 2019-02-11       0           0
 2019-02-12       0           0
 2019-02-13       1  

In [140]:
test_data[0]

{'start': '2010-01-04 00:00:00',
 'target': [-1,
  0,
  0,
  0,
  0,
  0,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  -1,
  -1,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  1,
  1,
  1,
  1,
  1,
  -1,
  -1,
  1,
  -1,
  -1,
  -1,
  1,
  1,
  1,
  1,
  1,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  1,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  1,
  1,
  1,
  1,
  1,
  -1,
  -1,
  -1,
  -1,
  1,
  1,
  -1,
  1,
  1,
  1,
  1,
  -1,
  -1,
  -1,
  1,
  1,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  1,
  -1,
  1,
  -1,
  -1,
  1,
  1,
  0,
  -1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 